In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
train = pd.read_csv('data/train.csv')
train['datetime'] = pd.to_datetime(train['datetime'], utc=True)
train['hour'] = train['datetime'].dt.hour

# Join electricity prices with training target by data_block_id and hour of day
electricity = pd.read_csv('data/electricity_prices.csv')
electricity['forecast_date'] = pd.to_datetime(electricity['forecast_date'], utc=True)
electricity['hour'] = electricity['forecast_date'].dt.hour
dataset = train.merge(electricity[['euros_per_mwh', 'hour', 'data_block_id']], how='left', on=['hour', 'data_block_id'])

# Join gas prices with training target by data_block_id
gas = pd.read_csv('data/gas_prices.csv')
dataset = dataset.merge(gas[['data_block_id', 'lowest_price_per_mwh', 'highest_price_per_mwh']], how='left', on='data_block_id')

# Join client types by data_block_id
client = pd.read_csv('data/client.csv')
client['data_block_id'] -= 2 # Client data_block_id seems to be ahead by 2 for the same days
dataset = dataset.merge(client.drop(columns = ['date']), how='left', on=['data_block_id', 'county', 'is_business', 'product_type'])

print(dataset.info())

dataset.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 15 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   county                 int64              
 1   is_business            int64              
 2   product_type           int64              
 3   target                 float64            
 4   is_consumption         int64              
 5   datetime               datetime64[ns, UTC]
 6   data_block_id          int64              
 7   row_id                 int64              
 8   prediction_unit_id     int64              
 9   hour                   int32              
 10  euros_per_mwh          float64            
 11  lowest_price_per_mwh   float64            
 12  highest_price_per_mwh  float64            
 13  eic_count              float64            
 14  installed_capacity     float64            
dtypes: datetime64[ns, UTC](1), float64(6), int32(1), int64(7)
memory u

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,hour,euros_per_mwh,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity
0,0,0,1,0.713,0,2021-09-01 00:00:00+00:00,0,0,0,0,NaN,NaN,NaN,108.0,952.89
1,0,0,1,96.590,1,2021-09-01 00:00:00+00:00,0,1,0,0,NaN,NaN,NaN,108.0,952.89
2,0,0,2,0.000,0,2021-09-01 00:00:00+00:00,0,2,1,0,NaN,NaN,NaN,17.0,166.40
3,0,0,2,17.314,1,2021-09-01 00:00:00+00:00,0,3,1,0,NaN,NaN,NaN,17.0,166.40
4,0,0,3,2.904,0,2021-09-01 00:00:00+00:00,0,4,2,0,NaN,NaN,NaN,688.0,7207.88


In [24]:
weather_map = pd.read_csv('input/weather_station_to_county_mapping.csv')
forecast_weather = pd.read_csv('data/forecast_weather.csv')
historical_weather = pd.read_csv('data/historical_weather.csv')



# Round longitude and latitude to nearest decimal
weather_map[['latitude', 'longitude']] = weather_map[['latitude', 'longitude']].astype(float).round(1)
forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)
historical_weather[['latitude', 'longitude']] = historical_weather[['latitude', 'longitude']].astype(float).round(1)

# Join county id to weather forecast via longitude and latitude
forecast_weather = forecast_weather.merge(weather_map[['longitude', 'latitude', 'county']], how='left', on=['longitude', 'latitude'])
historical_weather = historical_weather.merge(weather_map[['longitude', 'latitude', 'county']], how='left', on=['longitude', 'latitude'])

# Drop NaNs that come from stations without county mapping and missing entries in weather
forecast_weather.dropna(axis=0, inplace=True) 
historical_weather.dropna(axis=0, inplace=True)


### Prepare forecast weather for join with training data by forecast_datetime
forecast_weather['county'] = forecast_weather['county'].astype('int64')

# Dropping the columns we don't need
forecast_weather.drop(['origin_datetime', 'latitude','longitude', 'hours_ahead', 'data_block_id'], axis=1, inplace= True)

forecast_weather.rename(columns={'forecast_datetime': 'datetime'}, inplace= True)
forecast_weather['datetime']= pd.to_datetime(forecast_weather['datetime'], utc= True)

# Take the mean of forecasts within the same hour across all stations
forecast_weather_datetime = forecast_weather.groupby([forecast_weather['datetime'].dt.to_period('h')])[list(forecast_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
forecast_weather_datetime['datetime']= pd.to_datetime(forecast_weather_datetime['datetime'].dt.to_timestamp(), utc=True)

# Take the mean of forecasts of stations within the same county
forecast_weather_datetime_county= forecast_weather.groupby(['county',forecast_weather['datetime'].dt.to_period('h')])[list(forecast_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
forecast_weather_datetime_county['datetime']= pd.to_datetime(
forecast_weather_datetime_county['datetime'].dt.to_timestamp(), utc=True)


### Prepare historical weather for join with training data by data_block_id and hour of day
historical_weather['county'] = historical_weather['county'].astype('int64')

# Dropping the columns we don't need
historical_weather.drop(['latitude','longitude'], axis=1, inplace= True)

historical_weather['datetime']= pd.to_datetime(historical_weather['datetime'], utc= True)

# Take the mean of values within the same hour across all stations
historical_weather_datetime = historical_weather.groupby([historical_weather['datetime'].dt.to_period('h')])[list(historical_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
historical_weather_datetime['datetime']= pd.to_datetime(historical_weather_datetime['datetime'].dt.to_timestamp(), utc=True)

# Take the mean of forecasts of stations within the same county
historical_weather_datetime_county= historical_weather.groupby(['county',historical_weather['datetime'].dt.to_period('h')])[list(historical_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
historical_weather_datetime_county['datetime']= pd.to_datetime(
historical_weather_datetime_county['datetime'].dt.to_timestamp(), utc=True)

historical_weather_datetime_county.info()

/tmp/ipykernel_2783/3127724278.py:31: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  forecast_weather_datetime = forecast_weather.groupby([forecast_weather['datetime'].dt.to_period('h')])[list(forecast_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
/tmp/ipykernel_2783/3127724278.py:35: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  forecast_weather_datetime_county= forecast_weather.groupby(['county',forecast_weather['datetime'].dt.to_period('h')])[list(forecast_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
/tmp/ipykernel_2783/3127724278.py:49: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  historical_weather_datetime = historical_weather.groupby([historical_weather['datetime'].dt.to_period('h')])[list(historical_weather.drop(['county','datetime'], axis= 1).columns)].mean().reset_index()
/t

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229125 entries, 0 to 229124
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   county                  229125 non-null  int64              
 1   datetime                229125 non-null  datetime64[ns, UTC]
 2   temperature             229125 non-null  float64            
 3   dewpoint                229125 non-null  float64            
 4   rain                    229125 non-null  float64            
 5   snowfall                229125 non-null  float64            
 6   surface_pressure        229125 non-null  float64            
 7   cloudcover_total        229125 non-null  float64            
 8   cloudcover_low          229125 non-null  float64            
 9   cloudcover_mid          229125 non-null  float64            
 10  cloudcover_high         229125 non-null  float64            
 11  windspeed_10m           22

In [25]:
# Join weather data with training target
dataset = dataset.merge(forecast_weather_datetime_county, how='left', on=['datetime', 'county'])

historical_weather_datetime_county['hour']= historical_weather_datetime_county['datetime'].dt.hour
historical_weather_datetime_county.drop('datetime', axis= 1, inplace= True)
dataset = dataset.merge(historical_weather_datetime_county, how='left', on=['data_block_id', 'county', 'hour'])

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 41 columns):
 #   Column                             Dtype              
---  ------                             -----              
 0   county                             int64              
 1   is_business                        int64              
 2   product_type                       int64              
 3   target                             float64            
 4   is_consumption                     int64              
 5   datetime                           datetime64[ns, UTC]
 6   data_block_id                      int64              
 7   row_id                             int64              
 8   prediction_unit_id                 int64              
 9   hour                               int32              
 10  euros_per_mwh                      float64            
 11  lowest_price_per_mwh               float64            
 12  highest_price_per_mwh              float64